In [1]:
%matplotlib inline
from torchvision import utils as utils_pt
from torchvision import transforms
from basic_fcn import *
from dataloader import *
import torchvision
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import time

In [2]:
import torch
import numpy as np
from dataloader import useful_ids

def pixel_acc(pred, Y):
    pred = pred.cpu()
    Y = Y.cpu()
    
    Y_num = Y.numpy()
    pred_num = pred.numpy()
    return np.count_nonzero(np.logical_and((pred_num == Y_num), np.isin(Y_num, useful_ids, invert=True))), np.count_nonzero(np.isin(Y_num, useful_ids, invert=True))

def iou(pred, Y, class_id):
    pred = pred.cpu()
    Y = Y.cpu()
    
    pred = pred.squeeze(1)  # BATCH x 1 x H x W => BATCH x H x W
    
    intersection = np.count_nonzero(np.logical_and(pred == class_id, Y == class_id)) 
    union = np.count_nonzero(np.logical_or(pred == class_id, Y == class_id))
    
    return intersection, union  # We smooth our devision to avoid 0/0

In [3]:
transformations = transforms.Compose([
    transforms.RandomHorizontalFlip(),
#     transforms.ToTensor()
#     transforms.
])

train_dataset = CityScapesDataset(csv_file='./Data/train.csv', transforms=transformations)
val_dataset = CityScapesDataset(csv_file='./Data/val.csv', transforms=transformations)
test_dataset = CityScapesDataset(csv_file='./Data/test.csv', transforms=transformations)


_batch_size = 2
_num_workers = 4
iou_classes = [11, 20, 24, 26, 33]


_batch_size = 4
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=_batch_size,
                          num_workers=4,
                          shuffle=True)
val_loader = DataLoader(dataset=val_dataset,
                          batch_size=_batch_size,
                          num_workers=4,
                          shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=_batch_size,
                          num_workers=4,
                          shuffle=True)

In [4]:
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform_(m.weight.data)
        #torch.nn.init.xavier_uniform(m.bias.data)
        m.bias.data.zero_()
        
epochs     = 100
criterion = nn.CrossEntropyLoss()# Choose an appropriate loss function from https://pytorch.org/docs/stable/_modules/torch/nn/modules/loss.html
fcn_model = FCN(n_class=n_class)
# for name, params in fcn_model.named_parameters():
#     print(name, params.shape, params.requires_grad)
    
fcn_model.apply(init_weights)
#fcn_model = torch.load('best_model')


optimizer = optim.Adam(fcn_model.parameters(), lr=5e-3)

In [5]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    fcn_model = fcn_model.cuda()
    
def train():
    for epoch in range(epochs):
        
        ts = time.time()
        for iter, (X, tar, Y) in enumerate(train_loader):
            optimizer.zero_grad()

            if use_gpu:
                inputs = X.cuda()   # Move your inputs onto the gpu
                labels = Y.cuda() # Move your labels onto the gpu
            else:
                inputs, labels = X, Y# Unpack variables into inputs and labels
            outputs = fcn_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            if iter % 100 == 0:
                print("epoch{}, iter{}, loss: {}".format(epoch, iter, loss.item(), time.time() - ts))
        
        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        torch.save(fcn_model, 'best_model')

        val(epoch)
        fcn_model.train()


def val(epoch):
    fcn_model.eval()
    
    total_loss = 0
    acc_num = 0
    acc_den = 0
    iou_num = len(iou_classes)*[0] 
    iou_den = len(iou_classes)*[0] 
    #Complete this function - Calculate loss, accuracy and IoU for every epoch
    # Make sure to include a softmax after the output from your model
    for iter, (X_val, tar_val, Y_val) in enumerate(val_loader):
        if use_gpu:
            inputs_val = X_val.cuda()   # Move your inputs onto the gpu
            labels_val = Y_val.cuda() # Move your labels onto the gpu
        else:
            inputs_val, labels_val = X_val, Y_val# Unpack variables into inputs and labels

        outputs_val = fcn_model(inputs_val)
        
        _, preds = torch.max(outputs_val, 1)
        loss_val = criterion(outputs_val, labels_val)
        
        preds_cpu = preds
        Y_cpu = Y_val
        
        total_loss += loss_val
        
        nn, dd = pixel_acc(preds_cpu, Y_cpu)
        acc_num += nn
        acc_den += dd
        
        for iou_cc_idx in range(len(iou_classes)):
            i_nn, i_dd = iou(preds_cpu, Y_cpu, iou_classes[iou_cc_idx])
            iou_num[iou_cc_idx] += i_nn
            iou_den[iou_cc_idx] += i_dd
        
    print("Validation loss after epoch {}, , loss: {}".format(epoch, total_loss))
    print("Accuracy on val after epoch {}, accuracy: {}".format(epoch, 100*acc_num/acc_den))
    for iou_cc_idx in range(len(iou_classes)):
        print("IOU for class {} on val after epoch{}, iou: {}".format(ou_classes[iou_cc_idx], epoch, 100*iou_num[iou_cc_idx]/iou_den[iou_cc_idx]))
           
        
        #Add accuracy and IoU

def test():
    #Complete this function - Calculate accuracy and IoU 
    # Make sure to include a softmax after the output from your model
    for iter, (X_tst, tar_tst, Y_tst) in enumerate(test_loader):
        if use_gpu:
            inputs_tst = X_tst.cuda()   # Move your inputs onto the gpu
            labels_tst = Y_tst.cuda() # Move your labels onto the gpu
        else:
            inputs_tst, labels_tst = X_tst, Y_tst# Unpack variables into inputs and labels

        outputs_tst = fcn_model(inputs_tst)
        loss_tst = criterion(outputs_tst, labels_tst)
        print("Validation loss after epoch {}, , loss: {}".format(epoch, loss_tst.item()))
        #print("Accuracy on test after epoch {}, accuracy: {}".format(epoch, pixel_acc(outputs_tst, Y_tst)))
        #print("IOU on test after epoch{}".format(epoch, iou(outputs_tst, Y_tst)))
            
        
        #Add accuracy and IoU
              
    
if __name__ == "__main__":
    #val(0)  # show the accuracy before training
    train()

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 10.73 GiB total capacity; 8.14 GiB already allocated; 16.56 MiB free; 9.23 MiB cached)